##QSVM on LumA v LumB
We start off the process by installing qiskit to this Jupyter Notebook. The process consists of preparing normalized data to pass it into qiskit's QSVM function and then waiting for the results to be processed. 
The QSVM process is almost exactly the same as a normal SVM. It differs only in the feature map which as you probably guessed, is a Quantum Circuit in the QSVM process.

In [0]:
!pip install qiskit

     |████████████████████████████████| 6.7MB 2.6MB/s 
     |████████████████████████████████| 23.3MB 13.0MB/s 
     |████████████████████████████████| 163kB 38.6MB/s 
     |████████████████████████████████| 174kB 51.6MB/s 
     |████████████████████████████████| 1.9MB 47.9MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 768kB 43.0MB/s 
     |████████████████████████████████| 5.8MB 43.8MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 296kB 44.0MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 573kB 34.3MB/s 
     |████████████████████████████████| 26.0MB 131kB/s 
     |████████████████████████████████| 2.7MB 48.3MB/s 
  Created wheel for qiskit: filename=qiskit-0.19.3-cp36-none-any.whl size=2899 sha256=fad81f9578ed785c42a0fdfa763c77ccb31aaa6f73dfebf53d2923c8cd9ba6c0
  Stored in directory: /root/.cache/pip/wheels/6d/48/7e/b22ec709562f69e

In [0]:
import numpy as np
from qiskit import BasicAer
from qiskit.providers.aer.noise import *
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.components.feature_maps import SecondOrderExpansion
from qiskit.aqua.components.multiclass_extensions import (ErrorCorrectingCode,AllPairs,OneAgainstRest)
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.utils import get_feature_dimension
from qiskit import IBMQ
import qiskit
from qiskit.ml.datasets import *
from qiskit.circuit.library import ZZFeatureMap
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_curve, confusion_matrix
from sklearn.model_selection import cross_val_predict
import feather


##Data Prep and loading my IBMQ Account
The data that we're importing is from the 'Quantum Annealing versus Classical Machine Learning Algorithms' paper. The data is of a feather type, so we use pandas to read it in as a DataFrame.

In [0]:
IBMQ.save_account('75ed5c790fc03e4cc6937b8bcd170d2fc3f72cb9f931ebbed2e11c9124b20d2b65c99a0a8cbb69884306828181f5051ebc01a2f4e4f4ba54fce8481b1c12dcfa')

In [0]:
data_train = pd.read_feather('drive/My Drive/Public_Folder/data5_lumAB_train_normalized.feather')
data_test = pd.read_feather('drive/My Drive/Public_Folder/data5_lumAB_test_normalized.feather')

In [0]:
x_data = pd.DataFrame(data=data_train, columns = data_train.columns)
y_data = pd.DataFrame(data=data_train, columns = ['cancer'])

x_test = pd.DataFrame(data = data_test, columns = data_test.columns)
y_test = pd.DataFrame(data = data_test, columns = ['cancer'])

We apply PCA to the dataset because ibmq can't handle more than 32 features (Each Qubit is mapped onto one feature and the maximum qubits available to us on IBM's OpenQasmSimulator are 32).

In [0]:
pca = PCA(n_components = 10)
label_encoder = LabelEncoder()
y_data_labeless = label_encoder.fit_transform(y_data)
y_test_labeless = label_encoder.fit_transform(y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
x_data_labeless = x_data.drop(columns = ['cancer'], axis=1)
x_test_labeless = x_data.drop(columns = ['cancer'], axis=1)

In [0]:
x_data_pca = pca.fit_transform(x_data_labeless)
x_test_pca = pca.transform(x_test_labeless)

In [0]:
A = np.empty(5)
B = np.empty(5)
A_test = np.empty(5)
B_test = np.empty(5)
for x in range(len(y_data_labeless)):
  if y_data_labeless[x] == 0:
    A = np.vstack((A, x_data_pca[x]))
  else:
    B = np.vstack((B, x_data_pca[x]))
for x in range(len(y_test_labeless)):
  if y_test_labeless[x] == 0:
    A_test = np.vstack((A_test, x_test_pca[x]))
  else:
    B_test = np.vstack((B_test, x_test_pca[x]))
A = np.delete(A, 0, axis=0)
B = np.delete(B, 0, axis=0)
A_test = np.delete(A_test, 0, axis=0)
B_test = np.delete(B_test, 0, axis=0)
sample = {
      'A' : A,
      'B' : B
}
test = {
    'A' : A_test,
    'B' : B_test
}

In [0]:
qiskit.__version__

'0.14.1'

In [0]:
shots = 1024
seed = 10598
feature_dims = 5

provider = IBMQ.load_account()
#backend1 = provider.get_backend('ibmq_vigo')
noise_model = NoiseModel.from_backend(backend1)
feature_map = ZZFeatureMap(feature_dimension=feature_dims, reps=2, entanglement='linear')
qsvm = QSVM(feature_map, sample, test)

In [0]:
backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=10598, seed_transpiler=10598)

In [0]:
result = qsvm.run(quantum_instance, noise_model=noise_model)

In [0]:
print("Accuracy = ", result['testing_accuracy'] * 100)

Accuracy =  55.73770491803278
